# Compound Fitting with `SAMMY`

This Jupyter Notebook cell demonstrates the application of the `SAMMY` tool for fitting spectral transmission data using the `pleiades` utilities.

The transmission data utilized in this analysis originates from the [RPI measurement](https://doi.org/10.1016/j.anucene.2014.01.041) of enriched samples of [Eu-nat](https://www-nds.iaea.org/exfor/servlet/X4sGetSubent?subID=14382008) and [Eu-153](https://www-nds.iaea.org/exfor/servlet/X4sGetSubent?subID=14382007), which are documented on [EXFOR](https://www-nds.iaea.org/index-meeting-crp/CM-RF-2013/).

Europium comprises two stable isotopes, Eu-151 and Eu-153, with natural abundances of 47.8% and 52.2%, respectively. This example focuses on determining the abundances of these isotopes in both natural and enriched samples solely based on fitting the resonance transmission spectrum.

The analysis involves two main steps:

1. **Individual Isotope Fitting:** SAMMY is executed separately for each isotope. A simple configuration file is used to generate the resonance parameter file (.par file) employing resonance data directly sourced from ENDF8.

2. **Compound Isotopic Fitting:** The resonance spectra of both isotopes are combined to form an isotopic compound. SAMMY is then run again to fit the resonance spectrum and determine the most probable abundances of the isotopes.

In [1]:
%matplotlib inline
import pandas
import sys
sys.path.append("/sammy/Pleiades")
import pleiades

## Configuration File for a Single Isotope

In this section, we provide an overview of the essential parameters within the configuration file. The configuration file is structured according to SAMMY conventions and contains vital information for the conversion process. Below, we focus on the key parameters relevant to this example:

- **ELMNT:** Specifies the element's name, denoted as 'Eu_151'. This identifier is crucial for SAMMY to recognize the isotope.

- **AW:** Is the isotopic mass. If `pleiades` identifies the `ELMNT` name, it automatically retrieves the mass from the AME database. Therefore, this parameter can be  set as 'auto'.

- **EMIN, EMAX:** Define the energy range for the analysis, indicating the minimum and maximum energies considered.

- **Commands:** This section includes essential commands for the configuration and generation of the `.par` file. Of particular importance is the 'INPUT IS ENDF/B FILE' command, instructing SAMMY to locate the reference to resonance data in the ENDF file. A MAT number will be replaced during subsequent steps, based on the isotope name.

- **Fit Parameters:** Defines crucial parameters for the fitting process:
  - `TEMP:` Represents the temperature in Kelvin.
  - `DIST:` Signifies the TOF (Time of Flight) distance in meters.
  - `CRFN:` Indicates the channel radius (not crucial at this stage).
  - `THICK:` Denotes the target thickness in units of atoms/barn.
  - `CROSS:` Specifies the reaction type, set here to 'TRANSMISSION.'

[comment]: # (- **Spin Groups:** This section is optional for this step. Here, two spin groups for the isotopes are specified. SAMMY automatically extracts this data from ENDF.)

Below is an example of the configuration file structure:

```ini
[Card1]
TITLE = input to convert ENDF8 file into a SAMMY par file

[Card2]
ELMNT = Eu-151
AW = auto
EMIN = 0.001
EMAX = 98.8

[Card3]
commands = CHI_SQUARED,CSISRS,NO_SOLVE_BAYES,USE_ENDF_ENERGY,INPUT IS ENDF/B FILE

[Card5]
TEMP = 296.
DIST = 25.584

[Card7]
CRFN = 1.0
THICK = 0.0021

[Card8]
CROSS = TRANSMISSION 


we save this file as `config_Eu_151.ini` and use `pleiades` to parse the file

In [2]:

from pleiades import sammyInput

# read config files and write inp files
saminp = sammyInput.InputFile("config_Eu_151.ini").process().write("Eu_151.inp")

# update the element name to Eu_153 and write a new file. 
# The matnumber and atomic weight are updates automaically
saminp.data["Card2"]["elmnt"] = "Eu-153"
saminp.data["Card2"]["aw"] = "auto"
saminp.process().write("Eu_153.inp")


In [3]:
!cat Eu_151.inp

input to convert ENDF8 file into a SAMMY par file                               
Eu-151      150.9199    0.0010   98.8000
CHI SQUARED IS WANTEd
CSISRS
DO NOT SOLVE BAYES EQUATIONS
USE ENERGY RANGE FROm endf/b file
INPUT IS ENDF/B FILE MAT=6325

  296.0000   25.5840
    1.0000    0.0021
TRANSMISSION                                                                    



check the content of .inp files produced

In [4]:
!cat Eu_153.inp

input to convert ENDF8 file into a SAMMY par file                               
Eu-153      152.9212    0.0010   98.8000
CHI SQUARED IS WANTEd
CSISRS
DO NOT SOLVE BAYES EQUATIONS
USE ENERGY RANGE FROm endf/b file
INPUT IS ENDF/B FILE MAT=6331

  296.0000   25.5840
    1.0000    0.0021
TRANSMISSION                                                                    



## Run SAMMY with ENDF data to produce .par file
For this step we use the `pleiades.sammyRunner` utility function `run_endf`

In [5]:
from pleiades import sammyRunner

sammyRunner.run_endf(archivename="Eu_151",inpfile="Eu_151.inp",datafile="Eu_nat.dat")

FileNotFoundError: [Errno 2] No such file or directory: 'archive/Eu_151/SAMQUA.PAR'

In [13]:
ls archive/Eu_151

Eu_151.dat  Eu_151.inp  Eu_151.out  results/
